In [1]:
print("hello world")

hello world


In [2]:
import requests

response = requests.get("https://api.binance.com/api/v3/klines?symbol=BTCUSDT&interval=3m&limit=500")

data = response.json()



In [3]:
print(len(data))
print(data[0])

500
[1762094340000, '110417.94000000', '110455.57000000', '110385.93000000', '110455.57000000', '19.64289000', 1762094519999, '2168884.99062900', 5305, '15.75202000', '1739254.70723750', '0']


# Full Project 


- Depth chart for a cryptocurrency of the day 
- 24h candle stick chart with 1 minute intervals
- Overall row with a summary of the cryptos we'll be watching today 

In [1]:
# imports 
import logging
import os
from dotenv import load_dotenv
from datetime import datetime
import requests
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Global config
load_dotenv()

logging.basicConfig(
  level=logging.INFO,
  format='%(asctime)s | %(levelname)s | %(name)s| %(message)s',
  handlers=[
        logging.FileHandler(f"loading_logs_{datetime.now().strftime('%Y%m%d')}.log"),
        logging.StreamHandler()
  ]  
)


## Kline data

In [3]:
def fetch_klines_24hrs(symbol):
    try:
        logging.info(f'Starting to load klines ...')
        response = requests.get(f"https://api.binance.com/api/v3/klines?symbol={symbol}&interval=3m&limit=500")
        response.raise_for_status()
        data = response.json()
        return data
    except Exception as e:
        logging.info(f"there was a problem loading klines{e}")

# test the function
data = fetch_klines_24hrs("BTCUSDT")

print(len(data))
print(data[0])

2025-11-06 02:42:23,453 | INFO | root| Starting to load klines ...


500
[1762296300000, '101178.25000000', '101402.00000000', '101178.25000000', '101308.30000000', '73.67643000', 1762296479999, '7462473.41568180', 16653, '43.46018000', '4401381.25927620', '0']


In [5]:
def transform_load_klines_data(data, symbol, ranking):
    # put it into a data frame 
    logging.info('Loading into the data frame')
    df = pd.DataFrame(data)

    logging.info("Renaming the columns")
    # rename the existing columns to what they need to be
    df.columns = ['K_Open_Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'K_Close_Time', 'Quote_Asset_Volume', 
                    'Number_of_Trades', 'TB_Base_Volume', 'TB_Quote_Volume', 'Ignore']
    
    # add the necessary columns 
    logging.info("Adding the additional columns")
    df["symbol"] = symbol
    df['ranking'] = ranking
    df['time_collected'] = datetime.now()

    # removing uneccesary columns
    logging.info("dropping unncecessary columns")
    df= df.drop('Ignore', axis=1)

    # type issues 
    logging.info("changing the numeric columns to numeric datatypes")

    num_cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'Quote_Asset_Volume', 'TB_Base_Volume', 'TB_Quote_Volume']

    df[num_cols]= df[num_cols].apply(pd.to_numeric, axis=1)
    
    logging.info("Changing the time column to the time convention ")
    time_cols = ["K_Open_Time","K_Close_Time", 'time_collected']


    df[time_cols] = (
        df[time_cols]
        .apply(lambda col: pd.to_datetime(col, unit='ms', utc=True))
        .apply(lambda x: x.dt.to_pydatetime()) 
        )
    


    print(df.head())
    # logging.info("loading the data")

    # conn = os.getenv("CONN")

    # engine = create_engine(conn)

    # df.to_sql("Kline_data", engine, if_exists="append", index=False)\

    
   
    

transform_load_klines_data(data, "BTCUSDT", 1)


2025-11-06 02:42:45,189 | INFO | root| Loading into the data frame
2025-11-06 02:42:45,192 | INFO | root| Renaming the columns
2025-11-06 02:42:45,193 | INFO | root| Adding the additional columns
2025-11-06 02:42:45,196 | INFO | root| dropping unncecessary columns
2025-11-06 02:42:45,198 | INFO | root| changing the numeric columns to numeric datatypes
2025-11-06 02:42:45,293 | INFO | root| Changing the time column to the time convention 


                K_Open_Time       Open       High        Low      Close  \
0 2025-11-04 22:45:00+00:00  101178.25  101402.00  101178.25  101308.30   
1 2025-11-04 22:48:00+00:00  101308.31  101365.37  101007.00  101007.00   
2 2025-11-04 22:51:00+00:00  101007.01  101140.00  100900.00  101032.97   
3 2025-11-04 22:54:00+00:00  101032.96  101163.05  101000.00  101011.60   
4 2025-11-04 22:57:00+00:00  101011.61  101295.71  101011.60  101295.71   

     Volume                     K_Close_Time  Quote_Asset_Volume  \
0  73.67643 2025-11-04 22:47:59.999000+00:00        7.462473e+06   
1  95.64293 2025-11-04 22:50:59.999000+00:00        9.677562e+06   
2  86.65061 2025-11-04 22:53:59.999000+00:00        8.754030e+06   
3  62.07239 2025-11-04 22:56:59.999000+00:00        6.274048e+06   
4  92.62600 2025-11-04 22:59:59.999000+00:00        9.364290e+06   

   Number_of_Trades  TB_Base_Volume  TB_Quote_Volume   symbol  ranking  \
0             16653        43.46018     4.401381e+06  BTCUSDT     

/tmp/ipykernel_127386/2522827732.py:35: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  .apply(lambda x: x.dt.to_pydatetime())


## Order book

In [9]:
def fetch_OB(symbol):
    try:
        logging.info(f'Starting to load order book data ...')
        response = requests.get(f"https://api.binance.com/api/v3/depth?symbol={symbol}&limit=500")
        response.raise_for_status()
        data = response.json()
        return data
    except Exception as e:
        logging.info(f"there was a problem loading order book data{e}")

# test the function
data = fetch_OB("BTCUSDT")

print(len(data))
print(data)

2025-10-30 09:04:52,964 | INFO | root| Starting to load order book data ...


3
{'lastUpdateId': 79317652571, 'bids': [['110194.83000000', '4.68264000'], ['110194.76000000', '0.00010000'], ['110193.89000000', '0.00010000'], ['110193.88000000', '0.01795000'], ['110193.17000000', '0.00005000'], ['110193.00000000', '0.00005000'], ['110190.82000000', '0.00005000'], ['110188.26000000', '0.00250000'], ['110187.25000000', '0.00005000'], ['110186.76000000', '0.00010000'], ['110186.75000000', '0.26136000'], ['110186.71000000', '0.00363000'], ['110186.31000000', '0.00010000'], ['110186.30000000', '0.12325000'], ['110184.59000000', '0.00005000'], ['110184.58000000', '0.40659000'], ['110184.55000000', '0.00005000'], ['110184.54000000', '0.47820000'], ['110184.52000000', '0.26280000'], ['110184.51000000', '0.15871000'], ['110184.50000000', '0.00005000'], ['110184.26000000', '0.00005000'], ['110183.91000000', '0.00005000'], ['110183.85000000', '0.00250000'], ['110183.81000000', '0.31505000'], ['110183.59000000', '0.14204000'], ['110182.86000000', '0.00100000'], ['110182.51000

In [ ]:
def transform_load_OB_data(data:dict, symbol:str, ranking:int)-> None:
    # put it into a data frame 
    logging.info('Loading into the data frame')
    df_bids = pd.DataFrame(data["bids"], columns = ['price', 'quantity'] )
    df_bids["side"] = 'bids'

    df_asks = pd.DataFrame(data["asks"], columns = ['price', 'quantity'])
    df_asks["side"] = 'asks'
    

    df= pd.concat([df_asks, df_bids], ignore_index=True)
    
    # add the necessary columns 
    logging.info("Adding the additional columns")
    df["symbol"] = symbol
    df['ranking'] = ranking
    df['time_collected'] = datetime.now()
    
    # type issues 
    logging.info("changing the numeric columns to numeric datatypes")

    num_cols = ['price', 'quantity']

    df[num_cols]= df[num_cols].apply(pd.to_numeric, axis=1)
    
    logging.info("loading the data")


    engine = create_engine(conn)

    df.to_sql("order_book", engine, if_exists="append", index=False)

transform_load_OB_data(data, "BTCUSDT", 1)

2025-10-30 09:08:33,095 | INFO | root| Loading into the data frame
2025-10-30 09:08:33,100 | INFO | root| Adding the additional columns
2025-10-30 09:08:33,101 | INFO | root| changing the numeric columns to numeric datatypes


       price  quantity  side   symbol  ranking             time_collected
0  110194.84   2.42743  asks  BTCUSDT        1 2025-10-30 09:08:33.101528
1  110194.85   0.00040  asks  BTCUSDT        1 2025-10-30 09:08:33.101528
2  110194.92   0.00010  asks  BTCUSDT        1 2025-10-30 09:08:33.101528
3  110195.52   0.00010  asks  BTCUSDT        1 2025-10-30 09:08:33.101528
4  110195.53   0.00751  asks  BTCUSDT        1 2025-10-30 09:08:33.101528


# Recent trades 


In [8]:
def fetch_recent_trades(symbol):
    try:
        logging.info(f'Starting to load order book data ...')
        response = requests.get(f"https://api.binance.com/api/v3/trades?symbol={symbol}&limit=500")
        response.raise_for_status()
        data = response.json()
        return data
    except Exception as e:
        logging.info(f"there was a problem loading order book data{e}")

# test the function
data = fetch_recent_trades("BTCUSDT")

print(len(data))
print(data[0])

2025-11-03 18:38:05,582 | INFO | root| Starting to load order book data ...


500
{'id': 5423957319, 'price': '106366.00000000', 'qty': '0.00096000', 'quoteQty': '102.11136000', 'time': 1762184285468, 'isBuyerMaker': False, 'isBestMatch': True}


In [10]:
def transform_load_recent_trades(data:dict, symbol:str, ranking:int)-> None:
    # put it into a data frame 
    logging.info('Loading into the data frame')
    df = pd.DataFrame(data)

    # rename the columns to snake case
    df  = df.rename(columns={"quoteQty":"quote_qty", "isBuyerMaker": "is_buyer_maker", "isBestMatch":"is_best_match"})
    
    # add the necessary columns 
    logging.info("Adding the additional columns")
    df["symbol"] = symbol
    df['ranking'] = ranking
    df['time_collected'] = datetime.now()

  

    # type issues 
    logging.info("changing the numeric columns to numeric datatypes")

    num_cols = ['price', 'qty', 'quote_qty']

    df[num_cols]= df[num_cols].apply(pd.to_numeric, axis=1)
    
    logging.info("Changing the time column to the time convention ")
    time_cols = ["time"]


    df[time_cols] = df[time_cols].apply(
        lambda col: col.apply(lambda x: datetime.fromtimestamp(x / 1000.0)) 
        )
    print(df.head())
    # logging.info("loading the data")

    # engine = create_engine(conn)

    # df.to_sql("recent_trades", engine, if_exists="append", index=False)



    
   
    

transform_load_recent_trades(data, "BTCUSDT", 1)

2025-11-03 18:38:25,104 | INFO | root| Loading into the data frame
2025-11-03 18:38:25,110 | INFO | root| Adding the additional columns
2025-11-03 18:38:25,114 | INFO | root| changing the numeric columns to numeric datatypes
2025-11-03 18:38:25,175 | INFO | root| Changing the time column to the time convention 


           id     price      qty  quote_qty                    time  \
0  5423957319  106366.0  0.00096  102.11136 2025-11-03 18:38:05.468   
1  5423957320  106366.0  0.00045   47.86470 2025-11-03 18:38:05.468   
2  5423957321  106366.0  0.00104  110.62064 2025-11-03 18:38:05.468   
3  5423957322  106366.0  0.00020   21.27320 2025-11-03 18:38:05.468   
4  5423957323  106366.0  0.00055   58.50130 2025-11-03 18:38:05.468   

   is_buyer_maker  is_best_match   symbol  ranking             time_collected  
0           False           True  BTCUSDT        1 2025-11-03 18:38:25.113161  
1           False           True  BTCUSDT        1 2025-11-03 18:38:25.113161  
2           False           True  BTCUSDT        1 2025-11-03 18:38:25.113161  
3           False           True  BTCUSDT        1 2025-11-03 18:38:25.113161  
4           False           True  BTCUSDT        1 2025-11-03 18:38:25.113161  


## Top ticker stats 

In [25]:
def fetch_top5_24hr() -> tuple[list, list]:
    try:
        logging.info(f'Starting to load top 5  data ...')
        response = requests.get(f"https://api.binance.com/api/v3/ticker/24hr")
        response.raise_for_status()
        data = response.json()
        # Filter based on the usdt tethered cryptos  
        usdt_pairs = [
            item for item in data
            if item['symbol'].endswith('USDT') and float(item['quoteVolume']) > 1000000 and float(item['askQty']) > 0.0
            ]
        # sort the data
        sorted_data = sorted(
            usdt_pairs,
            key = lambda x: float(x["priceChangePercent"]),
            reverse=True
        )

        top_5 = [x['symbol'] for x in sorted_data[:5]]

        # push this to the other tasks with xcoms 
        return sorted_data[:5] , top_5
    except Exception as e:
        logging.info(f"there was a problem loading top 5 data{e}")


data, top_5 = fetch_top5_24hr()

print(len(data))
print(data[0])     

2025-10-30 09:19:01,686 | INFO | root| Starting to load top 5  data ...


5
{'symbol': 'PERPUSDT', 'priceChange': '0.04220000', 'priceChangePercent': '24.197', 'weightedAvgPrice': '0.19285179', 'prevClosePrice': '0.17450000', 'lastPrice': '0.21660000', 'lastQty': '125.45000000', 'bidPrice': '0.21640000', 'bidQty': '100.71000000', 'askPrice': '0.21660000', 'askQty': '128.19000000', 'openPrice': '0.17440000', 'highPrice': '0.22440000', 'lowPrice': '0.17190000', 'volume': '18939630.74000000', 'quoteVolume': '3652541.76527600', 'openTime': 1761718741364, 'closeTime': 1761805141364, 'firstId': 57897054, 'lastId': 57939579, 'count': 42526}


In [ ]:
def transform_load_top5_24hr(data:list) -> None:
    # put it into a data frame 
    logging.info('Loading into the data frame')
    df = pd.DataFrame(data)
    
    df = df.rename(columns={'priceChange':'price_change', 'priceChangePercent':'price_change_percent', 'weightedAvgPrice':'weighted_avg_price', 'prevClosePrice':'prev_close_price', 'lastPrice':'last_price', 'lastQty': 'last_qty','bidPrice':'bid_price', 'bidQty':'bid_qty', 'askPrice':'ask_price','askQty':'ask_qty', 'openPrice':'open_price', 'highPrice':'high_price','lowPrice':'low_price', 'quoteVolume':'quote_volume', "openTime":"open_time","closeTime":"close_time", "firstId":"first_id", "lastId":"last_id"})
    # add the necessary columns 
    logging.info("Adding the additional columns")
    df['time_collected'] = datetime.now()


    # type issues 
    logging.info("changing the numeric columns to numeric datatypes")

    num_cols = ['price_change', 'price_change_percent', 'weighted_avg_price', 'prev_close_price', 'last_price', 'last_qty','bid_price', 'bid_qty', 'ask_price','ask_qty', 'open_price', 'high_price','low_price', 'volume', 'quote_volume']

    df[num_cols]= df[num_cols].apply(pd.to_numeric, axis=1)
    
    logging.info("Changing the time column to the time convention ")
    time_cols = ["open_time","close_time"]


    df[time_cols] = df[time_cols].apply(
        lambda col: col.apply(lambda x: datetime.fromtimestamp(x / 1000.0)) 
        )
  
    logging.info("loading the data")

    engine = create_engine(conn)

    df.to_sql("top_24hr", engine, if_exists="append", index=False)


transform_load_top5_24hr(data)

2025-10-30 09:29:19,031 | INFO | root| Loading into the data frame
2025-10-30 09:29:19,034 | INFO | root| Adding the additional columns
2025-10-30 09:29:19,036 | INFO | root| changing the numeric columns to numeric datatypes
2025-10-30 09:29:19,043 | INFO | root| Changing the time column to the time convention 


      symbol  price_change  price_change_percent  weighted_avg_price  \
0   PERPUSDT        0.0422                24.197            0.192852   
1    KDAUSDT        0.0073                17.340            0.047161   
2   LQTYUSDT        0.0660                12.571            0.569767   
3  LUMIAUSDT        0.0160                12.030            0.153199   
4     OGUSDT        1.2090                11.370           11.336568   

   prev_close_price  last_price  last_qty  bid_price   bid_qty  ask_price  \
0            0.1745      0.2166    125.45     0.2164    100.71     0.2166   
1            0.0420      0.0494   4469.61     0.0494   7876.60     0.0496   
2            0.5260      0.5910    316.40     0.5910    856.80     0.5920   
3            0.1330      0.1490   3574.63     0.1480  28460.87     0.1490   
4           10.6320     11.8420     23.10    11.8400      2.30    11.8500   

   ...  high_price  low_price        volume  quote_volume  \
0  ...      0.2244     0.1719  1.893963e+07

In [36]:
import requests
import json

# Step 1: Fetch all 24hr ticker data
url = "https://api.binance.com/api/v3/ticker/24hr"
response = requests.get(url)
data = response.json()

# Step 2: Filter for USDT pairs and high volume (optional: >$1M volume)
usdt_pairs = [
    item for item in data
    if item['symbol'].endswith('USDT') and float(item['quoteVolume']) > 1000000 and float(item['askQty']) > 0.0]

# Step 3: Sort by priceChangePercent descending and take top 5
top_gainers = sorted(usdt_pairs, key=lambda x: float(x['priceChangePercent']), reverse=True)[:5]

# Step 4: Print results
print("Top 5 Cryptos by 24h % Gain:")
for coin in top_gainers:
    symbol = coin['symbol']
    percent = coin['priceChangePercent']
    price = coin['lastPrice']
    print(f"{symbol}: +{percent}% (Price: ${price}) with latest order ask {coin['askPrice'] , }")


Top 5 Cryptos by 24h % Gain:
FUNUSDT: +16.941% (Price: $0.00411400) with latest order ask ('0.00411400',)
HBARUSDT: +11.811% (Price: $0.20448000) with latest order ask ('0.20448000',)
TRUMPUSDT: +9.943% (Price: $6.91100000) with latest order ask ('6.91200000',)
AMPUSDT: +8.187% (Price: $0.00277500) with latest order ask ('0.00277100',)
ADXUSDT: +7.098% (Price: $0.12070000) with latest order ask ('0.12060000',)
